In [1]:
# Example from https://github.com/JuliaPOMDP/POMDPs.jl testing out my environment

In [2]:
using POMDPs, QuickPOMDPs, POMDPModelTools, POMDPSimulators, QMDP

m = QuickPOMDP(
    states = ["left", "right"],
    actions = ["left", "right", "listen"],
    observations = ["left", "right"],
    initialstate = Uniform(["left", "right"]),
    discount = 0.95,

    transition = function (s, a)
        if a == "listen"
            return Deterministic(s) # tiger stays behind the same door
        else # a door is opened
            return Uniform(["left", "right"]) # reset
        end
    end,

    observation = function (s, a, sp)
        if a == "listen"
            if sp == "left"
                return SparseCat(["left", "right"], [0.85, 0.15]) # sparse categorical distribution
            else
                return SparseCat(["right", "left"], [0.85, 0.15])
            end
        else
            return Uniform(["left", "right"])
        end
    end,

    reward = function (s, a)
        if a == "listen"
            return -1.0
        elseif s == a # the tiger was found
            return -100.0
        else # the tiger was escaped
            return 10.0
        end
    end
)

solver = QMDPSolver()
policy = solve(solver, m)

rsum = 0.0
for (s,b,a,o,r) in stepthrough(m, policy, "s,b,a,o,r", max_steps=10)
    println("s: $s, b: $([pdf(b,s) for s in states(m)]), a: $a, o: $o")
    global rsum += r
end
println("Undiscounted reward was $rsum.")

s: right, b: [0.5, 0.5], a: listen, o: right
s: right, b: [0.15, 0.85], a: listen, o: right
s: right, b: [0.0302013422818792, 0.9697986577181208], a: left, o: left
s: right, b: [0.5, 0.5], a: listen, o: right
s: right, b: [0.15, 0.85], a: listen, o: right
s: right, b: [0.0302013422818792, 0.9697986577181208], a: left, o: right
s: left, b: [0.5, 0.5], a: listen, o: left
s: left, b: [0.85, 0.15], a: listen, o: left
s: left, b: [0.9697986577181208, 0.0302013422818792], a: right, o: right
s: right, b: [0.5, 0.5], a: listen, o: right
Undiscounted reward was 23.0.
